### **DEMO 1**

### El problema más simple (con Google OR Tools)

In [ ]:
from ortools.sat.python.cp_model import CpModel, CpSolver

m = CpModel()

x = m.new_int_var(0, 50, "x")
y = m.new_int_var(0, 50, "y")

m.add( x+ 2*y <= 55 )

m.maximize(30*x+15*y)

s = CpSolver()
s.Solve(m)

print(f"obj = {s.objective_value}")
print(f"x = {s.value(x)}")
print(f"y = {s.value(y)}")

### SUDOKU (sin gráficos)

In [ ]:
from ortools.sat.python.cp_model import CpModel, CpSolver

m = CpModel()

# defino la matriz
k = [[m.new_int_var(0, 9, "x" + str(i) + str(j)) for i in range(9)] for j in range(9)]

# cargo los numeros base
m.add(k[0][0] == 5)
m.add(k[0][1] == 3)
m.add(k[0][4] == 7)
m.add(k[1][0] == 6)
m.add(k[1][3] == 1)
m.add(k[1][4] == 9)
m.add(k[1][5] == 5)
m.add(k[2][1] == 9)
m.add(k[2][2] == 8)
m.add(k[2][7] == 6)
m.add(k[3][0] == 8)
m.add(k[3][4] == 6)
m.add(k[3][8] == 3)
m.add(k[4][0] == 4)
m.add(k[4][3] == 8)
m.add(k[4][5] == 3)
m.add(k[4][8] == 1)
m.add(k[5][0] == 7)
m.add(k[5][4] == 2)
m.add(k[5][8] == 6)
m.add(k[6][1] == 6)
m.add(k[6][6] == 2)
m.add(k[6][7] == 8)
m.add(k[7][3] == 4)
m.add(k[7][4] == 1)
m.add(k[7][5] == 9)
m.add(k[7][8] == 5)
m.add(k[8][4] == 8)
m.add(k[8][7] == 7)
m.add(k[8][8] == 9)

# filas diferentes
for i in range(9):
    m.add_all_different([k[i][j] for j in range(9)])

# columnas diferentes
for j in range(9):
    m.add_all_different([k[i][j] for i in range(9)])

# diferente cuadrante

for x in range(0, 9, 3):
    for y in range(0, 9, 3):
        m.add_all_different([k[i][j] for i in range(x, x + 3) for j in range(y, y + 3)])

s = CpSolver()
s.Solve(m)

for i in k:
    for v in i:
            if s.Value(v):
                print(v, '=', s.Value(v))


### SUDOKU (con gráficos)

In [ ]:
from sys import stdout
import numpy as np
import matplotlib.pyplot as plt


# función para gráficar el sudoku
def draw_grid(orig, values, name):
    stdout.write(name)
    stdout.write(":\n")    
    fig, ax = plt.subplots(figsize =(4,4))
    min_val, max_val = 0, 9
    R =  range(0,9)
    for l in R:
        for c in R:
            o = orig[c][l]
            v = values[c][l]
            s = " "
            if v > 0:
                if o > 0:
                    s = str(v)  # es valor fijo
                    color = 'red'
                else:
                    s = str(v)  # no es valor fijo
                    color = 'black'

            ax.text(l+0.5,8.5-c, s, color = color , va='center', ha='center')
        ax.set_xlim(min_val, max_val)
    ax.set_ylim(min_val, max_val)
    ax.set_xticks(np.arange(max_val))
    ax.set_yticks(np.arange(max_val))
    ax.grid()
    plt.show()

In [ ]:
p1 = [[5, 3, 0, 0, 7, 0, 0, 0, 0],
      [6, 0, 0, 1, 9, 5, 0, 0, 0],
      [0, 9, 8, 0, 0, 0, 0, 6, 0],
      [8, 0, 0, 0, 6, 0, 0, 0, 3],
      [4, 0, 0, 8, 0, 3, 0, 0, 1],
      [7, 0, 0, 0, 2, 0, 0, 0, 6],
      [0, 6, 0, 0, 0, 0, 2, 8, 0],
      [0, 0, 0, 4, 1, 9, 0, 0, 5],
      [0, 0, 0, 0, 8, 0, 0, 7, 9],            
      ]

draw_grid(p1, p1, "PROBLEMA INICIAL")

In [ ]:
from ortools.sat.python.cp_model import CpModel, CpSolver

m = CpModel()

# defino la matriz
k = [[m.new_int_var(1, 9, "x" + str(i) + str(j)) for i in range(9)] for j in range(9)]

# cargo los numeros base
for i in range(9):
    for j in range(9):
        if p1[i][j]>0: m.add(k[i][j] == p1[i][j])

# filas diferentes
for i in range(9):
    m.add_all_different([k[i][j] for j in range(9)])

# columnas diferentes
for j in range(9):
    m.add_all_different([k[i][j] for i in range(9)])

# diferente cuadrante
for x in range(0, 9, 3):
    for y in range(0, 9, 3):
        m.add_all_different([k[i][j] for i in range(x, x + 3) for j in range(y, y + 3)])

s = CpSolver()
e = s.solve(m)

if e == 4 or e == 2:       # 4: OPTIMAL, 2: FEASIBLE
    sol = [[s.value(k[l][c]) for c in range(9)] for l in range(9)]
    draw_grid(p1, sol, "Problema resuelto")
else:
    stdout.write("No se encontró solución\n")